# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 2 2022 2:50PM,249921,10,0086139711,ISDIN Corporation,Released
1,Nov 2 2022 2:50PM,249921,10,0086139712,ISDIN Corporation,Released
2,Nov 2 2022 2:50PM,249921,10,0086139713,ISDIN Corporation,Released
3,Nov 2 2022 2:50PM,249921,10,0086139722,ISDIN Corporation,Released
4,Nov 2 2022 2:50PM,249921,10,0086139723,ISDIN Corporation,Released
5,Nov 2 2022 2:50PM,249921,10,0086139725,ISDIN Corporation,Released
6,Nov 2 2022 2:50PM,249921,10,0086139728,ISDIN Corporation,Released
7,Nov 2 2022 2:50PM,249921,10,0086139726,ISDIN Corporation,Released
8,Nov 2 2022 2:50PM,249921,10,0086139727,ISDIN Corporation,Released
9,Nov 2 2022 2:43PM,249920,10,Enova-10921,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,249914,Released,3
40,249916,Released,1
41,249919,Released,1
42,249920,Released,4
43,249921,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249914,NaN,NaN,3.0
249916,NaN,NaN,1.0
249919,NaN,NaN,1.0
249920,NaN,NaN,4.0
249921,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249828,0.0,0.0,1.0
249837,0.0,0.0,2.0
249846,5.0,8.0,2.0
249855,0.0,11.0,2.0
249856,25.0,1.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249828,0,0,1
249837,0,0,2
249846,5,8,2
249855,0,11,2
249856,25,1,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249828,0,0,1
1,249837,0,0,2
2,249846,5,8,2
3,249855,0,11,2
4,249856,25,1,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249828,,,1
1,249837,,,2
2,249846,5,8,2
3,249855,,11,2
4,249856,25,1,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 2 2022 2:50PM,249921,10,ISDIN Corporation
9,Nov 2 2022 2:43PM,249920,10,Emerginnova
13,Nov 2 2022 2:13PM,249919,19,"AdvaGen Pharma, Ltd"
14,Nov 2 2022 1:51PM,249916,10,Yusen – 3D Matrix
15,Nov 2 2022 1:48PM,249914,10,"Methapharm, Inc."
18,Nov 2 2022 1:41PM,249911,21,"NBTY Global, Inc."
19,Nov 2 2022 1:34PM,249908,10,ISDIN Corporation
27,Nov 2 2022 1:31PM,249907,19,"AdvaGen Pharma, Ltd"
39,Nov 2 2022 1:30PM,249906,21,"NBTY Global, Inc."
40,Nov 2 2022 1:04PM,249902,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 2 2022 2:50PM,249921,10,ISDIN Corporation,,,9
1,Nov 2 2022 2:43PM,249920,10,Emerginnova,,,4
2,Nov 2 2022 2:13PM,249919,19,"AdvaGen Pharma, Ltd",,,1
3,Nov 2 2022 1:51PM,249916,10,Yusen – 3D Matrix,,,1
4,Nov 2 2022 1:48PM,249914,10,"Methapharm, Inc.",,,3
5,Nov 2 2022 1:41PM,249911,21,"NBTY Global, Inc.",,,1
6,Nov 2 2022 1:34PM,249908,10,ISDIN Corporation,,,8
7,Nov 2 2022 1:31PM,249907,19,"AdvaGen Pharma, Ltd",,,12
8,Nov 2 2022 1:30PM,249906,21,"NBTY Global, Inc.",,,1
9,Nov 2 2022 1:04PM,249902,10,ISDIN Corporation,,2,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 2:50PM,249921,10,ISDIN Corporation,9,,
1,Nov 2 2022 2:43PM,249920,10,Emerginnova,4,,
2,Nov 2 2022 2:13PM,249919,19,"AdvaGen Pharma, Ltd",1,,
3,Nov 2 2022 1:51PM,249916,10,Yusen – 3D Matrix,1,,
4,Nov 2 2022 1:48PM,249914,10,"Methapharm, Inc.",3,,
5,Nov 2 2022 1:41PM,249911,21,"NBTY Global, Inc.",1,,
6,Nov 2 2022 1:34PM,249908,10,ISDIN Corporation,8,,
7,Nov 2 2022 1:31PM,249907,19,"AdvaGen Pharma, Ltd",12,,
8,Nov 2 2022 1:30PM,249906,21,"NBTY Global, Inc.",1,,
9,Nov 2 2022 1:04PM,249902,10,ISDIN Corporation,8,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 2:50PM,249921,10,ISDIN Corporation,9,,
1,Nov 2 2022 2:43PM,249920,10,Emerginnova,4,,
2,Nov 2 2022 2:13PM,249919,19,"AdvaGen Pharma, Ltd",1,,
3,Nov 2 2022 1:51PM,249916,10,Yusen – 3D Matrix,1,,
4,Nov 2 2022 1:48PM,249914,10,"Methapharm, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 2:50PM,249921,10,ISDIN Corporation,9,NaN,NaN
1,Nov 2 2022 2:43PM,249920,10,Emerginnova,4,NaN,NaN
2,Nov 2 2022 2:13PM,249919,19,"AdvaGen Pharma, Ltd",1,NaN,NaN
3,Nov 2 2022 1:51PM,249916,10,Yusen – 3D Matrix,1,NaN,NaN
4,Nov 2 2022 1:48PM,249914,10,"Methapharm, Inc.",3,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 2:50PM,249921,10,ISDIN Corporation,9,0.0,0.0
1,Nov 2 2022 2:43PM,249920,10,Emerginnova,4,0.0,0.0
2,Nov 2 2022 2:13PM,249919,19,"AdvaGen Pharma, Ltd",1,0.0,0.0
3,Nov 2 2022 1:51PM,249916,10,Yusen – 3D Matrix,1,0.0,0.0
4,Nov 2 2022 1:48PM,249914,10,"Methapharm, Inc.",3,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3498380,43,42.0,11.0
15,499693,5,1.0,25.0
16,499772,2,0.0,0.0
19,499826,13,0.0,0.0
20,249900,1,0.0,0.0
21,2998709,12,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3498380,43,42.0,11.0
1,15,499693,5,1.0,25.0
2,16,499772,2,0.0,0.0
3,19,499826,13,0.0,0.0
4,20,249900,1,0.0,0.0
5,21,2998709,12,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,43,42.0,11.0
1,15,5,1.0,25.0
2,16,2,0.0,0.0
3,19,13,0.0,0.0
4,20,1,0.0,0.0
5,21,12,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,43.0
1,15,Released,5.0
2,16,Released,2.0
3,19,Released,13.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,11.0,25.0,0.0,0.0,0.0,0.0
Executing,42.0,1.0,0.0,0.0,0.0,0.0
Released,43.0,5.0,2.0,13.0,1.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,11.0,25.0,0.0,0.0,0.0,0.0
1,Executing,42.0,1.0,0.0,0.0,0.0,0.0
2,Released,43.0,5.0,2.0,13.0,1.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,11.0,25.0,0.0,0.0,0.0,0.0
1,Executing,42.0,1.0,0.0,0.0,0.0,0.0
2,Released,43.0,5.0,2.0,13.0,1.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()